# LSTM Recurrent Neural Network

Tutorial/example [here](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/)

## compatibility: tensorflow 2.0.0

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from tqdm import tqdm

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 6} ) 
sess = tf.Session(config=config) 
# keras.backend.set_session(sess)

# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18002718370936039762
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11420538375592745374
physical_device_desc: "device: XLA_CPU device"
]


In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [5]:
# load ascii text and covert to lowercase
filename = "data.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [6]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in tqdm(enumerate(chars)))

In [7]:
print('#unique chars: \t', len(chars))
print('Total chars: \t', len(raw_text))

#unique chars: 	 1440
Total chars: 	 629491742


In [ ]:
from tqdm import tqdm

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
n_chars = len(raw_text)
n_vocab = len(chars)

print("balk")

for i in tqdm(range(0, n_chars - seq_length, 1)):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

print ("Total Patterns: ", n_patterns)

  0%|                                                                                      | 17720/629491642 [00:00<2:07:06, 82540.13it/s]

balk


  7%|█████▋                                                                            | 43691632/629491642 [17:17<1:19:57, 122114.44it/s]

In [ ]:
print('Example dataX (len =', len(dataX[1]), '):\n\n', dataX[0], '\n')

int_to_char = dict((i, c) for i, c in tqdm(enumerate(chars)))
seq_in = [int_to_char[value] for value in tqdm(dataX[0])]

print(seq_in)

In [ ]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# define the checkpoint
filepath="models\weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# load the network weights
filename = "models\weights-improvement-20-1.8710.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)